In [30]:
import numpy as np
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt.modeling import op, variable, dot
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [118]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist('C:/Users/RE-Giorgio/Documents/OptimusPrime/Data', kind='train')


indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:100,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:100].astype('float64') 

yLabel2[:] = +1
yLabel4[:] = -1

X = np.concatenate([xLabel2, xLabel4])
y = np.concatenate([yLabel2, yLabel4])

scaler = StandardScaler()
X=scaler.fit_transform(X)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1696995) 

In [221]:
class Svm:
    
    def __init__(self, X, y, gamma, C):
        
        self.X = X
        self.y = y
        self.alpha = np.random.normal(0,1,(1,self.X.shape[0]))
        self.b = np.random.randint(1)
        self.C = C
        self.gamma = gamma
        
    def predict(self,X, y):
        
        z = ((self.alpha*y) @ self.kernel_gauss(X)) + self.b
        a = np.sign(z)    
        return a
        
    def kernel_gauss(self, X):
        
        split = np.split(X, X.shape[0])
        res = []
        for array in split:
            res.append(np.exp(-self.gamma * (np.sum(np.square(array-X), axis = 1))))
        return np.array(res)
    
    def kernel_poly(self,x):
        
        return np.squeeze((np.split(x, x.shape[0]) @ np.stack([x.T]*x.shape[0]) + 1)**self.gamma)
    
        
    def optimize(self):
        
        y = self.y.reshape(-1,1) 
        H = y*self.kernel_gauss(self.X)*y 
        P = cvxopt_matrix(H)
        q = cvxopt_matrix(-np.ones((self.X.shape[0], 1)))
        G = cvxopt_matrix(np.vstack((np.eye(self.X.shape[0])*-1,np.eye(self.X.shape[0]))))
        h = cvxopt_matrix(np.hstack((np.zeros(self.X.shape[0]), np.ones(self.X.shape[0]) * self.C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        
        res = cvxopt_solvers.qp(P, q, G, h, A, b)
        
        alpha = np.array(res['x'])
        self.alpha = alpha.T
        
        
        
# ====================== to be fixed ====================== #
        
#         idx = (alpha.ravel() > 1e-3)
#         w = self.X.T @ (y*alpha)
#         b = (1 - (y[idx, :]*(self.X[idx,:]) @ w))/y[idx, :]
#         self.b = b[0]
        
        
svm = Svm(X_train, y_train, gamma = 6, C = 4)
svm.optimize()
y_pred = svm.predict(X_train, y_train)
print(accuracy_score(y_train.reshape(-1,1), y_pred.reshape(-1,1)))

     pcost       dcost       gap    pres   dres
 0: -4.4443e+02 -8.6752e+03  8e+03  9e-14  9e-16
 1: -7.4235e+02 -1.3535e+03  6e+02  2e-13  2e-16
 2: -7.9977e+02 -8.2834e+02  3e+01  3e-13  8e-17
 3: -7.9998e+02 -8.0027e+02  3e-01  3e-14  1e-17
 4: -7.9998e+02 -7.9998e+02  3e-03  2e-13  3e-17
 5: -7.9998e+02 -7.9998e+02  3e-05  2e-13  4e-17
 6: -7.9998e+02 -7.9998e+02  3e-07  1e-13  5e-17
 7: -7.9998e+02 -7.9998e+02  3e-09  2e-13  5e-17
 8: -7.9998e+02 -7.9998e+02  3e-11  2e-13  3e-17
Optimal solution found.
0.5025


In [160]:
y_train.shape

(1600,)

In [119]:

class Svm:
    
    def __init__(self, X, y, gamma, C):
        
        self.X = X
        self.y = y
        self.alpha = np.random.normal(0,1,(1,self.X.shape[0]))
        self.b = np.random.normal(0,1,(1,self.X.shape[0]))
        self.C = C
        self.gamma = gamma
        
    def predict(self,X, y):
        
        z = ((self.alpha*y) @ self.kernel_poly(X)) + self.b
        a = np.sign(z)    
        return a
        
    def kernel_gauss(self, X):
        
        radius = np.split(X, X.shape[0]) - X
        return np.exp(-self.gamma * (np.linalg.norm(radius, axis = 0)))
    
    def kernel_poly(self,x):
        
        return np.squeeze((np.split(x, x.shape[0]) @ np.stack([x.T]*x.shape[0]) + 1)**self.gamma)
    
    def obj_fun(self, alpha):
        
        Y = np.identity(self.y.shape[0])*self.y
        return 0.5 * np.linalg.norm((Y @ alpha) @ self.X )**2 - np.ones_like(self.alpha) @ alpha
        
     
    def con_1(self, x):
        return self.y @ x.T
    def con_2(self, x):
            return x - self.C
        
    def optimize(self):
                             
        cons = [{'type':'eq', 'fun': self.con_1},
        {'type':'ineq', 'fun': self.con_2}]

        res = minimize(self.obj_fun, x0 = self.alpha,method = "SLSQP", constraints = cons)
        self.alpha = res.x   
        
svm = Svm(X_train, y_train, gamma = 6, C = 4)
svm.optimize()
y_pred = svm.predict(X_train, y_train)
print(accuracy_score(y_train.reshape(-1,1), y_pred.reshape(-1,1)))

KeyboardInterrupt: 

In [116]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
print(accuracy_score(y_train.reshape(-1,1), y_pred.reshape(-1,1)))

0.9425


In [150]:
X = np.arange(30).reshape(10,3)
a = []
for array in np.split(X, 10):
    a.append(np.sum((array[0] - X)**2, axis = 1))
print(a)

[array([   0,   27,  108,  243,  432,  675,  972, 1323, 1728, 2187],
      dtype=int32), array([  27,    0,   27,  108,  243,  432,  675,  972, 1323, 1728],
      dtype=int32), array([ 108,   27,    0,   27,  108,  243,  432,  675,  972, 1323],
      dtype=int32), array([243, 108,  27,   0,  27, 108, 243, 432, 675, 972], dtype=int32), array([432, 243, 108,  27,   0,  27, 108, 243, 432, 675], dtype=int32), array([675, 432, 243, 108,  27,   0,  27, 108, 243, 432], dtype=int32), array([972, 675, 432, 243, 108,  27,   0,  27, 108, 243], dtype=int32), array([1323,  972,  675,  432,  243,  108,   27,    0,   27,  108],
      dtype=int32), array([1728, 1323,  972,  675,  432,  243,  108,   27,    0,   27],
      dtype=int32), array([2187, 1728, 1323,  972,  675,  432,  243,  108,   27,    0],
      dtype=int32)]


In [142]:
np.linalg.norm(np.split(X,10)-np.stack([X]*10), axis = 2)**2

array([[   0.,   27.,  108.,  243.,  432.,  675.,  972., 1323., 1728.,
        2187.],
       [  27.,    0.,   27.,  108.,  243.,  432.,  675.,  972., 1323.,
        1728.],
       [ 108.,   27.,    0.,   27.,  108.,  243.,  432.,  675.,  972.,
        1323.],
       [ 243.,  108.,   27.,    0.,   27.,  108.,  243.,  432.,  675.,
         972.],
       [ 432.,  243.,  108.,   27.,    0.,   27.,  108.,  243.,  432.,
         675.],
       [ 675.,  432.,  243.,  108.,   27.,    0.,   27.,  108.,  243.,
         432.],
       [ 972.,  675.,  432.,  243.,  108.,   27.,    0.,   27.,  108.,
         243.],
       [1323.,  972.,  675.,  432.,  243.,  108.,   27.,    0.,   27.,
         108.],
       [1728., 1323.,  972.,  675.,  432.,  243.,  108.,   27.,    0.,
          27.],
       [2187., 1728., 1323.,  972.,  675.,  432.,  243.,  108.,   27.,
           0.]])

In [195]:
np.random.seed(1)
X[np.where(np.random.normal(0,1,(10)) >0),].shape

(1, 4, 3)